## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from configg import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Min Temp,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Port Alfred,-33.5906,26.8910,68.18,68.18,70,1,7.00,ZA,clear sky
1,1,Nantucket,41.2835,-70.0995,55.02,57.31,85,100,16.11,US,overcast clouds
2,2,Prince Albert,53.2001,-105.7677,35.55,35.55,64,75,16.11,CA,broken clouds
3,3,Vaini,-21.2000,-175.2000,73.56,73.56,68,20,5.75,TO,few clouds
4,4,Berlevag,70.8578,29.0864,22.80,24.75,86,75,19.57,NO,light snow


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temp you prefer for travel?"))
max_temp = float(input("What is the maximum temp you prefer for travel?"))

What is the minimum temp you prefer for travel?70
What is the maximum temp you prefer for travel?80


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

preferred_cities_df = city_data_df.loc[(city_data_df["Min Temp"] >= min_temp) & 
                                       (city_data_df["Max Temp"] <= max_temp)]

preferred_cities_df


,City_ID,City,Lat,Lng,Min Temp,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
3,3,Vaini,-21.2000,-175.2000,73.56,73.56,68,20,5.75,TO,few clouds
9,9,Cape Town,-33.9258,18.4232,73.90,75.61,42,0,8.05,ZA,clear sky
15,15,Isangel,-19.5500,169.2667,75.78,75.78,73,97,12.95,VU,overcast clouds
19,19,Bredasdorp,-34.5322,20.0403,72.52,72.52,39,5,10.20,ZA,clear sky
27,27,Atuona,-9.8000,-139.0333,78.28,78.28,77,35,22.28,PF,light rain
...,...,...,...,...,...,...,...,...,...,...,...
661,661,Puerto Penasco,31.3167,-113.5333,70.92,70.92,72,0,17.00,MX,clear sky
666,666,Aitape,-3.1374,142.3475,79.81,79.81,87,60,4.45,PG,broken clouds
667,667,Bailesti,44.0167,23.3500,74.95,74.95,41,5,3.65,RO,clear sky
668,668,Kangshan,28.8812,116.4311,70.72,70.72,85,94,7.99,CN,overcast clouds


In [6]:
# 4a. Determine if there are any empty rows.

preferred_cities_df.isnull()

,City_ID,City,Lat,Lng,Min Temp,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
3,False,False,False,False,False,False,False,False,False,False,False
9,False,False,False,False,False,False,False,False,False,False,False
15,False,False,False,False,False,False,False,False,False,False,False
19,False,False,False,False,False,False,False,False,False,False,False
27,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...
661,False,False,False,False,False,False,False,False,False,False,False
666,False,False,False,False,False,False,False,False,False,False,False
667,False,False,False,False,False,False,False,False,False,False,False
668,False,False,False,False,False,False,False,False,False,False,False


In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
if preferred_cities_df.isnull().values.any() == True:
    preferred_cities_df = preferred_cities_df.dropna()
clean_df = preferred_cities_df

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Vaini,TO,73.56,few clouds,-21.2000,-175.2000,
9,Cape Town,ZA,75.61,clear sky,-33.9258,18.4232,
15,Isangel,VU,75.78,overcast clouds,-19.5500,169.2667,
19,Bredasdorp,ZA,72.52,clear sky,-34.5322,20.0403,
27,Atuona,PF,78.28,light rain,-9.8000,-139.0333,
35,Rikitea,PF,77.07,broken clouds,-23.1203,-134.9692,
58,Saint-Philippe,RE,77.22,light rain,-21.3585,55.7679,
62,Sao Filipe,CV,74.71,clear sky,14.8961,-24.4956,
69,Sibolga,ID,79.32,very heavy rain,1.7427,98.7792,
72,Bambous Virieux,MU,79.05,light rain,-20.3428,57.7575,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"

    # 6e. Make request and retrieve the JSON data from the search. 
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print(f"Hotel not found... Skipping.{index}")
        

Hotel not found... Skipping.90
Hotel not found... Skipping.319
Hotel not found... Skipping.414
Hotel not found... Skipping.432
Hotel not found... Skipping.458
Hotel not found... Skipping.666
Hotel not found... Skipping.668


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))